In [1]:
import json
import numpy as np
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim

## Data loader

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

class NPYDataset(Dataset):
    def __init__(self, main_directory):
        super().__init__()
        self.main_directory = main_directory
        self.filenames = self._load_filenames()

    def _load_filenames(self):
        # Traverse subdirectories to find all .npy files
        filenames = []
        for root, _, files in os.walk(self.main_directory):
            for file in files:
                if file.endswith('.npy'):
                    full_path = os.path.join(root, file)
                    filenames.append(full_path)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        file_path = self.filenames[idx]
        data = np.load(file_path, allow_pickle=True)
        keypoints = np.ascontiguousarray(data['data'], dtype=np.float32)
        label = np.float32(data['label'])
        # Convert to tensors
        keypoints = torch.tensor(keypoints)
        label = torch.tensor(label)
        return keypoints, label


In [3]:


# DataLoader setup
data_directory = 'splitted_datapoints_phase/train'
dataset = NPYDataset(data_directory)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

val_dataset = NPYDataset('splitted_datapoints_phase/val')
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [4]:
len(dataloader)

2691

In [5]:
len(val_loader)

674

In [6]:
for keypoints, labels in dataloader:
    # keypoints = keypoints.squeeze(0)
    print(keypoints.shape, labels)

torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[1.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[2.]])
torch.Size([1, 1, 7, 33, 2]) tensor([[0.]])
torch.Size([1, 1, 7, 33, 2]) ten

In [7]:
# len(val_loader)

## Model

In [6]:
class ConvLSTMModel(nn.Module):
    def __init__(self, input_shape, hidden_dim, num_classes):
        super(ConvLSTMModel, self).__init__()
        self.segment_length, self.keypoints, self.coords = input_shape
        
        self.conv1d = nn.Conv1d(in_channels=self.keypoints * self.coords, out_channels=32, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(input_size=32, hidden_size=hidden_dim, batch_first=True)
        # Change to output a single value for binary classification
        self.fc = nn.Linear(hidden_dim, num_classes) 
         
    def forward(self, x):
        x = x.view(x.size(0), self.keypoints * self.coords, self.segment_length)
        x = self.conv1d(x)
        x = torch.relu(x)
        x = x.transpose(1, 2)
        x, (hn, cn) = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        # x = torch.softmax(x)  # Sigmoid activation for binary classification
        return x


In [7]:

# Assume 'model' is an instance of your model class
model = ConvLSTMModel(input_shape=(7, 33, 2), hidden_dim=64, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [8]:
# Define a function for evaluating the model on the validation set
def evaluate(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():  # No need to compute gradients for validation
        for keypoints, labels in val_loader:
            # Forward pass
            outputs = model(keypoints)  # outputs shape should be [batch_size, 1]
            outputs = outputs.squeeze()
            labels = labels.squeeze().long()
            # Compute loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted_labels = torch.max(outputs.data, 0)
            # predicted_labels = outputs.round()  # Round the outputs to get the final predictions for binary classification
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += 1
    
    # Calculate average loss and accuracy
    avg_loss = running_loss / len(val_loader)
    accuracy = correct_predictions / total_predictions
    return avg_loss, accuracy

In [9]:
import torch
from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard
writer = SummaryWriter(log_dir='./runs/convLSTM_phase')
num_epochs = 500
# Variables to keep track of the best validation accuracy
best_val_acc = 0.0
best_model_path = 'phase_classifier_weights/best_model.pth'
last_model_path = 'phase_classifier_weights/last_model.pth'
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for keypoints, labels in dataloader:
        optimizer.zero_grad()
        # Forward pass
        outputs = model(keypoints)  # outputs shape should be [batch_size, 1]
        outputs = outputs.squeeze()
        labels = labels.squeeze().long()
        # print('outputs: ', outputs)
        # print('labels: ', labels)
        # Compute loss
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # print('outputs', outputs)
        # Calculate accuracy
        _, predicted_labels = torch.max(outputs.data, 0)
        # predicted_labels = outputs.round()  # Round the outputs to get the final predictions for binary classification
        # print('predicted_labels',predicted_labels, 'True labels:', labels)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += 1

        # Calculate accuracy and loss for the epoch
    accuracy = correct_predictions / total_predictions * 100
    epoch_loss = running_loss / len(dataloader)

    # Perform validation
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    
    # Log loss and accuracy to TensorBoard
    writer.add_scalar('Loss/train', epoch_loss, epoch)
    writer.add_scalar('Accuracy/train', accuracy, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    
    # Print validation metrics
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.2f}%')
    print(f'Epoch {epoch+1}/{num_epochs} - Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Save the last model weights
    torch.save(model.state_dict(), last_model_path)
    
    # Save the best model weights based on validation accuracy
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f'Best model saved at epoch {epoch+1} with val acc: {val_accuracy:.4f}')

print(f'Best validation accuracy: {best_val_acc:.4f}')
# Close the TensorBoard writer
writer.close()


Epoch [1/500], Train Loss: 1.0217, Train Accuracy: 45.89%
Epoch 1/500 - Validation Loss: 1.0141, Validation Accuracy: 0.4481
Best model saved at epoch 1 with val acc: 0.4481
Epoch [2/500], Train Loss: 1.0134, Train Accuracy: 44.70%
Epoch 2/500 - Validation Loss: 1.0211, Validation Accuracy: 0.4481
Epoch [3/500], Train Loss: 1.0169, Train Accuracy: 44.30%
Epoch 3/500 - Validation Loss: 1.0128, Validation Accuracy: 0.4481
Epoch [4/500], Train Loss: 1.0134, Train Accuracy: 43.92%
Epoch 4/500 - Validation Loss: 1.0123, Validation Accuracy: 0.4481
Epoch [5/500], Train Loss: 1.0130, Train Accuracy: 44.85%
Epoch 5/500 - Validation Loss: 1.0121, Validation Accuracy: 0.4481
Epoch [6/500], Train Loss: 1.0134, Train Accuracy: 44.41%
Epoch 6/500 - Validation Loss: 1.0122, Validation Accuracy: 0.4481
Epoch [7/500], Train Loss: 1.0128, Train Accuracy: 44.85%
Epoch 7/500 - Validation Loss: 1.0125, Validation Accuracy: 0.4481
Epoch [8/500], Train Loss: 1.0126, Train Accuracy: 44.26%
Epoch 8/500 - Vali

In [13]:
import torch
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Define a function for evaluating the model on the validation set
def evaluate_with_confusion_matrix(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():  # No need to compute gradients for validation
        for keypoints, labels in val_loader:
            # Forward pass
            outputs = model(keypoints)  # outputs shape should be [batch_size, 1]
            outputs = outputs.squeeze()
            labels = labels.squeeze().long()
            # Compute loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            # Get predictions
            _, predicted_labels = torch.max(outputs.data, 0)  # This should be along dimension 1 for batch data
            all_labels.append(labels)
            all_predictions.append(predicted_labels.tolist())
    
    # Calculate average loss
    avg_loss = running_loss / len(val_loader)
    
    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    # Optionally calculate accuracy from the confusion matrix
    accuracy = np.trace(cm) / np.sum(cm)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('f1-score: ', f1)
    return avg_loss, accuracy, cm

# Use the function
avg_loss, accuracy, cm = evaluate_with_confusion_matrix(model, val_loader, criterion)
print(f"Average Loss: {avg_loss}, Accuracy: {accuracy}")
print("Confusion Matrix:\n", cm)


Average Loss: 0.6931243710031938, Accuracy: 0.7166172106824926
Confusion Matrix:
 [[258   0  12]
 [ 69   0  33]
 [ 77   0 225]]


In [1]:
258/(258+12)

0.9555555555555556

In [3]:
225/(225+77)

0.7450331125827815

## Model simple LSTM

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define LSTM classifier model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [9]:
# # Generate sample data
# X_train = np.random.rand(100, 10, 50)  # 100 sequences of length 10 with 50 features each
# y_train = np.random.randint(2, size=(100,))  # Binary labels (0 or 1)

# # Convert data to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Define model parameters
input_size = 33*2
hidden_size = 64
num_layers = 2
output_size = 3


In [14]:
# Instantiate the model
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [15]:
import torch
from torch.utils.tensorboard import SummaryWriter

# Initialize TensorBoard
writer = SummaryWriter(log_dir='./runs/LSTM_phase')
num_epochs = 500
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    
    for keypoints, labels in dataloader:
        # Squeeze unnecessary dimensions
        keypoints = keypoints.squeeze(0)  # Now keypoints shape should be [1, 20, 17, 2]
        labels = labels.squeeze(0)        # Now labels shape should be [1]
        # print('keypoints:',keypoints.shape,'labels:', labels.shape)
        
        optimizer.zero_grad()
        outputs = model(keypoints)
        # print('outputs before',outputs, 'labels before',labels)

        outputs = outputs.squeeze()
        labels = labels.squeeze().long()
        # print('outputs after',outputs, 'labels after',labels)
        # break
        
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 0)
        # Total number of labels
        total += 1
        
        # Total correct predictions
        correct += (predicted == labels).sum().item()
    
    # Calculate accuracy and loss for the epoch
    accuracy = correct / total * 100
    epoch_loss = running_loss / len(dataloader)
    
    # Log loss and accuracy to TensorBoard
    writer.add_scalar('Loss/train', epoch_loss, epoch)
    writer.add_scalar('Accuracy/train', accuracy, epoch)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Close the TensorBoard writer
writer.close()


Epoch [1/500], Loss: 0.9545, Accuracy: 55.00%
Epoch [2/500], Loss: 0.8755, Accuracy: 61.87%
Epoch [3/500], Loss: 0.8177, Accuracy: 65.89%
Epoch [4/500], Loss: 0.8080, Accuracy: 66.26%
Epoch [5/500], Loss: 0.7994, Accuracy: 67.67%
Epoch [6/500], Loss: 0.8222, Accuracy: 64.62%
Epoch [7/500], Loss: 0.7966, Accuracy: 67.60%
Epoch [8/500], Loss: 0.9528, Accuracy: 50.46%
Epoch [9/500], Loss: 1.0183, Accuracy: 43.07%
Epoch [10/500], Loss: 1.0160, Accuracy: 43.85%
Epoch [11/500], Loss: 1.0150, Accuracy: 43.40%
Epoch [12/500], Loss: 0.9076, Accuracy: 59.01%
Epoch [13/500], Loss: 0.7877, Accuracy: 69.38%
Epoch [14/500], Loss: 0.7720, Accuracy: 69.04%
Epoch [15/500], Loss: 0.7869, Accuracy: 68.97%
Epoch [16/500], Loss: 0.7614, Accuracy: 70.12%
Epoch [17/500], Loss: 0.7779, Accuracy: 69.45%
Epoch [18/500], Loss: 0.7775, Accuracy: 69.42%
Epoch [19/500], Loss: 0.7558, Accuracy: 70.27%
Epoch [20/500], Loss: 0.7570, Accuracy: 70.61%
Epoch [21/500], Loss: 0.7711, Accuracy: 68.52%
Epoch [22/500], Loss: 